In [1]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import *


In [2]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)

# Actually import
import_opt = {'num_slices':33, 'alpha':0.15}

In [70]:
ref_frames, all_matches = track_via_reference_frames(btf_fname_red, num_reference_frames=2)

  0%|                                                                     | 0/2 [00:00<?, ?it/s]

Building reference frames...


  0%|                                                                     | 0/2 [00:00<?, ?it/s]

Pairwise matching all reference frames...
Matched neuron 0 to 103 based on 68 features
Matched neuron 1 to 88 based on 24 features
Matched neuron 2 to 70 based on 6 features
Matched neuron 3 to 75 based on 38 features
Matched neuron 4 to 116 based on 13 features
Matched neuron 5 to 83 based on 104 features
Could not match neuron 6
Matched neuron 7 to 77 based on 7 features
Matched neuron 8 to 104 based on 8 features
Could not match neuron 9
Matched neuron 10 to 87 based on 35 features
Matched neuron 11 to 85 based on 23 features
Matched neuron 12 to 78 based on 16 features
Matched neuron 13 to 103 based on 73 features
Matched neuron 14 to 98 based on 22 features
Matched neuron 15 to 105 based on 79 features
Matched neuron 16 to 86 based on 21 features
Matched neuron 17 to 104 based on 5 features
Could not match neuron 18
Could not match neuron 19
Matched neuron 20 to 16 based on 8 features
Matched neuron 21 to 99 based on 6 features
Matched neuron 22 to 7 based on 6 features
Matched ne

100%|█████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  9.43it/s]

Matched neuron 72 to 14 based on 7 features
Matched neuron 73 to 95 based on 46 features
Matched neuron 74 to 10 based on 17 features
Matched neuron 75 to 3 based on 33 features
Matched neuron 76 to 0 based on 32 features
Matched neuron 77 to 7 based on 8 features
Matched neuron 78 to 12 based on 15 features
Could not match neuron 79
Matched neuron 80 to 31 based on 8 features
Could not match neuron 81
Matched neuron 82 to 6 based on 9 features
Matched neuron 83 to 5 based on 36 features
Matched neuron 84 to 8 based on 7 features
Matched neuron 85 to 11 based on 18 features
Matched neuron 86 to 16 based on 31 features
Matched neuron 87 to 10 based on 20 features
Matched neuron 88 to 1 based on 11 features
Matched neuron 89 to 5 based on 76 features
Matched neuron 90 to 13 based on 9 features
Could not match neuron 91
Matched neuron 92 to 25 based on 20 features
Matched neuron 93 to 32 based on 19 features
Matched neuron 94 to 30 based on 21 features
Could not match neuron 95
Matched ne

In [65]:
%debug

> c:\users\charles.fieseler\documents\current_work\dlc_for_wbfm\dlc_for_wbfm\utils\feature_detection\feature_pipeline.py(253)calc_2frame_matches_using_class()
    251         for f0 in this_f0:
    252             #i_match = np.argwhere(feature_matches_ind[0,:]==f0)
--> 253             i_match = feature_matches_dict[f0]
    254             print(f"Feature 0 {f0} matched with index {i_match}")
    255             #if len(i_match) > 0:

ipdb> f0
array([15], dtype=int64)
ipdb> f0[0]
15
ipdb> feature_matches_dict[f0[0]]
3346
ipdb> q


In [44]:
this_f0 = ref_frames[0].get_features_of_neuron(0)
feature_matches = np.array([[14, 2],[2,3]])

In [47]:
for f0 in this_f0:
    i_match = np.argwhere(feature_matches[0,:]==f0)
    print(i_match, f0)

[] [12]
[] [13]
[[0]] [14]
[] [71]
[] [73]
[] [86]
[] [87]
[] [123]
[] [133]
[] [134]
[] [167]
[] [168]
[] [187]
[] [191]
[] [192]
[] [211]
[] [216]
[] [232]
[] [233]
[] [237]
[] [253]
[] [257]
[] [272]
[] [284]
[] [286]
[] [287]
[] [288]
[] [290]
[] [292]
[] [297]
[] [357]
[] [361]
[] [405]
[] [407]
[] [450]
[] [452]
[] [483]
[] [484]
[] [509]
[] [510]
[] [534]
[] [537]
[] [540]
[] [559]
[] [562]
[] [582]
[] [585]
[] [654]
[] [656]
[] [704]
[] [705]
[] [707]
[] [710]
[] [739]
[] [767]
[] [770]
[] [794]
[] [796]
[] [817]
[] [819]
[] [840]
[] [861]
[] [941]
[] [993]
[] [1029]
[] [1058]
[] [1088]
[] [1112]
[] [1137]
[] [1161]
[] [1167]
[] [1233]
[] [1237]
[] [1283]
[] [1287]
[] [1326]
[] [1361]
[] [1390]
[] [1415]
[] [1438]
[] [1459]
[] [1462]
[] [1466]
[] [1536]
[] [1544]
[] [1604]
[] [1607]
[] [1652]
[] [1656]
[] [1686]
[] [1719]
[] [1747]
[] [1778]
[] [1809]
[] [1813]
[] [1933]
[] [1936]
[] [2010]
[] [2014]
[] [2077]
[] [2079]
[] [2124]
[] [2157]
[] [2192]
[] [2223]
[] [2274]
[] [2391

In [ ]:
import cv2
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import *
im = get_single_volume(btf_fname_red, 0, num_slices=33)[0,...]

In [ ]:
im.shape
kp, d = detect_features(im, 1000)

In [ ]:
kp[0].pt



In [ ]:
dat = get_single_volume(btf_fname_red, 0, num_slices=33)
dat.shape[1:]

In [ ]:
cv2.KeyPoint.convert(kp2[0,...])
#kp3 = cv2.KeyPoint_convert(kp2)

In [11]:
a = [1, 3, 5, 7]
b = [3, 5, 6]

In [13]:
[a.index(val) if val in a for val in b]


SyntaxError: invalid syntax (<ipython-input-13-1bcc70044f20>, line 1)

In [ ]:
a[0,0]